In [ ]:
RESOURCES_PATH = '../../../resources'

In [ ]:
import json
import pickle
from pathlib import Path
import pandas as pd
import numpy as np
from IPython.display import display

In [ ]:
def get_model_reports(model):
    report_paths = [f for f in Path(f'./{model}/report').iterdir()]

    result = {}
    for report_path in report_paths:
        with open(report_path, 'r') as fin:
            result[report_path.stem] = json.load(fin)
    
    return result

def get_reports():
     models = [str(f) for f in Path('./').iterdir() if f.is_dir()]

     result = {}
     for model in models:
         model_reports = get_model_reports(model)
         for report_name in model_reports:
             result[f'{model} {report_name}'] = model_reports[report_name]
    
     return result

In [ ]:
reports = get_reports()

## Common report

In [ ]:
common_df = pd.DataFrame(reports).T.set_index('Name')
common_df

In [ ]:
common_df.sort_values(by='[Cleared Test] Log Loss')[['[Cleared Test] Log Loss']] # Cleared test model rating

In [ ]:
common_df.sort_values(by='[Original Test] Log Loss')[['[Original Test] Log Loss']] # Original test model rating

## Confusion Matrix

In [ ]:
with open(f'{RESOURCES_PATH}/dataset/turnover/label_encoder.pkl', 'rb') as fin:
    le = pickle.load(fin)

In [ ]:
def get_cat_most_likely_mistakes(conf_mtx, cat):
    total_mistakes = conf_mtx[cat].sum() - conf_mtx[cat][cat]
    mistakes = np.concatenate((conf_mtx[cat][:cat], np.zeros(1), conf_mtx[cat][cat+1:]))
    mistakes_norm = mistakes / total_mistakes
    top_mistakes = sorted(enumerate(mistakes_norm), key=lambda i: i[1], reverse=True)[:3]

    result = ''
    for i in range(len(top_mistakes)):
        result += f'    {i+1}. {le.inverse_transform([top_mistakes[i][0]])[0][:50]} - {round(top_mistakes[i][1] * 100, 2)}%\n'
    
    return result 

In [ ]:
def get_most_confused(conf_mtx):
    conf_mtx = np.array(conf_mtx)
    cat_total = conf_mtx.sum(axis=1)
    cat_correct = conf_mtx.diagonal()
    cat_errors = cat_total - cat_correct
    cat_errors_norm = cat_errors / cat_errors.sum()
    cat_errors_in_cat_norm = cat_errors / cat_total
    top_wrong = sorted(enumerate(cat_errors_norm), key=lambda i: i[1], reverse=True)[:10]

    result = ''
    for i in range(len(top_wrong)):
        result += f'{i+1}. {le.inverse_transform([top_wrong[i][0]])[0][:50]} - {round(top_wrong[i][1] * 100, 2)}% ({round(cat_errors_in_cat_norm[top_wrong[i][0]] * 100, 2)}% in cat.)\n'
        result += get_cat_most_likely_mistakes(conf_mtx, top_wrong[i][0])
        result += '\n'
    
    return result 

In [ ]:
with open("./bert/confusion_matrix/base.json", "r") as fin:
    ft_bert_fc_conf_mtx = json.load(fin)

with open("./mean_fc_nn/confusion_matrix/fasttext.json", "r") as fin:
    fasttext_fc_conf_mtx = json.load(fin)

with open("./mean_fc_nn/confusion_matrix/bert.json", "r") as fin:
    bert_fc_conf_mtx = json.load(fin)

### Cleared Test

In [ ]:
df = pd.DataFrame({
    'Fine-Tunned BERT FC': {'': get_most_confused(ft_bert_fc_conf_mtx['Cleared Test'])},
    'fastText FC': {'': get_most_confused(fasttext_fc_conf_mtx['Cleared Test'])},
    'BERT FC': {'': get_most_confused(bert_fc_conf_mtx['Cleared Test'])}
})

display(df.style.set_properties(**{
    'text-align': 'left',
    'white-space': 'pre-wrap'
}))

### Original Test

In [ ]:
df = pd.DataFrame({
    'Fine-Tunned BERT FC': {'': get_most_confused(ft_bert_fc_conf_mtx['Original Test'])},
    'fastText FC': {'': get_most_confused(fasttext_fc_conf_mtx['Original Test'])},
    'BERT FC': {'': get_most_confused(bert_fc_conf_mtx['Original Test'])}
})

display(df.style.set_properties(**{
    'text-align': 'left',
    'white-space': 'pre-wrap'
}))